In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from src.dataset import pca_data_loader
from tqdm import tqdm

ModuleNotFoundError: No module named 'src'

In [26]:
# 加载数据
train_loader, test_loader = pca_data_loader()

# 设置设备（如果有 GPU 可用则使用 GPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
# 定义多层感知机模型（可配置层数和节点数）
class CustomMLP(nn.Module):
    def __init__(self, input_size=10, hidden_layers=[50, 50], output_size=10):
        super(CustomMLP, self).__init__()
        layers = []
        in_features = input_size
        
        # 添加隐藏层
        for hidden_units in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_units))
            layers.append(nn.ReLU())
            in_features = hidden_units
        
        # 添加输出层
        layers.append(nn.Linear(in_features, output_size))
        
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [28]:
# 初始化模型、损失函数和优化器
hidden_layers = [200, 100, 50, 25, 10, 20, 40]  # 可以根据需要调整隐藏层数量和每层的节点数
model = CustomMLP(input_size=10, hidden_layers=hidden_layers, output_size=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [29]:
# 训练模型
epochs = 10
with tqdm(total=epochs * len(train_loader), desc="Training Progress", unit='batch') as pbar:
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            print("inputs requires_grad:", inputs.requires_grad)

            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.update(1)

Training Progress: 100%|██████████| 9380/9380 [00:31<00:00, 299.53batch/s]


In [30]:
# 在测试集上评估模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 89.58%
